In [1]:
import json

with open('test-set-imda.json') as fopen:
    data = json.load(fopen)

In [2]:
import malaya_speech
import speech_recognition as sr

r = sr.Recognizer()

In [3]:
import unicodedata
import re
import itertools

vocabs = [" ", "a", "e", "n", "i", "t", "o", "u", "s", "k", "r", "l", "h", "d", "m", "g", "y", "b", "p", "w", "c", "f", "j", "v", "z", "0", "1", "x", "2", "q", "5", "3", "4", "6", "9", "8", "7"]

def preprocessing_text(string):
        
    string = unicodedata.normalize('NFC', string.lower())
    string = ''.join([c if c in vocabs else ' ' for c in string])
    string = re.sub(r'[ ]+', ' ', string).strip()
    string = (
        ''.join(''.join(s)[:2] for _, s in itertools.groupby(string))
    )
    return string

In [4]:
data[0]

['test-wav/221930043.WAV', 'b r sreenivasan ho see beng and sai hua kuan']

In [5]:
from tqdm import tqdm

wer, cer = [], []
for i in tqdm(range(len(data))):
    try:
        with sr.AudioFile(data[i][0]) as source:
            a = r.record(source)

        text = r.recognize_google(a, language = 'en-SG')
        text = preprocessing_text(text)
        
        wer.append(malaya_speech.metrics.calculate_wer(data[i][1], text))
        cer.append(malaya_speech.metrics.calculate_cer(data[i][1], text))
    except Exception as e:
        print(e)

  5%|▌         | 827/15189 [18:33<3:15:33,  1.22it/s] 

 31%|███       | 4643/15189 [1:44:57<2:57:35,  1.01s/it]

 31%|███       | 4671/15189 [1:45:33<3:12:04,  1.10s/it]

 46%|████▌     | 6936/15189 [2:37:31<2:58:09,  1.30s/it]

 49%|████▉     | 7414/15189 [2:47:46<2:34:16,  1.19s/it]

...

 65%|██████▌   | 9913/15189 [3:44:05<1:34:38,  1.08s/it]

 71%|███████   | 10789/15189 [4:04:30<1:29:24,  1.22s/it]

 84%|████████▍ | 12784/15189 [4:49:35<1:01:36,  1.54s/it]

 85%|████████▍ | 12854/15189 [4:51:13<37:46,  1.03it/s]  

100%|██████████| 15189/15189 [5:38:46<00:00,  1.34s/it]  


In [6]:
import numpy as np
np.mean(wer), np.mean(cer)

(0.49413491638393103, 0.30262968615506136)